<a href="https://colab.research.google.com/github/akshaygrao77/DeepLearning-Assignment3/blob/main/DL_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [2]:
!curl https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar --output daksh.tar
!tar -xvf  'daksh.tar' 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1915M  100 1915M    0     0  86.4M      0  0:00:22  0:00:22 --:--:--  208M
dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script

In [9]:
def obtain_input_target_data_from_path(path,tokenizer_obj):
  input_texts = []
  target_texts = []
  
  df = pd.read_csv(path,sep="\t",names=["1", "2","3"]).astype(str)
  if tokenizer_obj is None:
    # Shuffle rows in random order with a fixed seed(for reproducability)
    df=df.sample(frac=1,random_state=1)
  # Add all the  input and target texts with start sequence and end sequence added to target 
  for index, row in df.iterrows():
      input_text=row['2']
      target_text= row['1']
      # Skip empty lines/words
      if target_text =='</s>' or input_text=='</s>':
        continue
      
      target_text = "\t" + target_text + "\n"
      input_texts.append(input_text)
      target_texts.append(target_text)
  
  return input_texts, target_texts

In [4]:
def convert_text_to_sequences(tokenizer_obj,inp_texts):
  if tokenizer_obj is None:
    tokenizer_obj = tf.keras.preprocessing.text.Tokenizer(filters='', char_level=True)
    tokenizer_obj.fit_on_texts(inp_texts)
  ret_tensor = tokenizer_obj.texts_to_sequences(inp_texts)
  ret_tensor = tf.keras.preprocessing.sequence.pad_sequences(ret_tensor,padding='post')

  return ret_tensor,tokenizer_obj

In [10]:
# This method converts a dataset(from path) to input and target sequences
def pre_process_data(path,input_tokenizer=None,target_tokenizer=None,input_length=None,target_length=None):
  
  input_texts, target_texts = obtain_input_target_data_from_path(path,input_tokenizer)
  
  input_tensor,input_tokenizer = convert_text_to_sequences(input_tokenizer,input_texts)
  
  target_tensor,target_tokenizer = convert_text_to_sequences(target_tokenizer,target_texts)
  
  # Above functions return padded version wrt longest sequence in the given list of sequence
  # The below function, pads more zeros wrt input_length and target_length
  if input_length is not None and target_length is not None:
      input_tensor=tf.concat([input_tensor,tf.zeros((input_tensor.shape[0],input_length-input_tensor.shape[1]))],axis=1)
      target_tensor=tf.concat([target_tensor,tf.zeros((target_tensor.shape[0],target_length-target_tensor.shape[1]))],axis=1)
  return input_texts,input_tensor,input_tokenizer,target_texts,target_tensor,target_tokenizer

In [6]:
transliteration_target_language = 'kn'

In [11]:
train_input_texts,train_input_tensor,input_tokenizer,train_target_texts,train_target_tensor,target_tokenizer = pre_process_data("/content/dakshina_dataset_v1.0/"+transliteration_target_language+"/lexicons/"+transliteration_target_language+".translit.sampled.train.tsv")
# Only training dataset is used to fit the tokenizer on text. Other datasets just use this vocab for pre-processing
# The length for padding is also set from training datasets
val_input_texts,val_input_tensor,val_input_tokenizer,val_target_texts,val_target_tensor,val_target_tokenizer = pre_process_data("/content/dakshina_dataset_v1.0/"+transliteration_target_language+"/lexicons/"+transliteration_target_language+".translit.sampled.dev.tsv",input_tokenizer,target_tokenizer,train_input_tensor.shape[1],train_target_tensor.shape[1])
test_input_texts,test_input_tensor,test_input_tokenizer,test_target_texts,test_target_tensor,test_target_tokenizer = pre_process_data("/content/dakshina_dataset_v1.0/"+transliteration_target_language+"/lexicons/"+transliteration_target_language+".translit.sampled.test.tsv",input_tokenizer,target_tokenizer,train_input_tensor.shape[1],train_target_tensor.shape[1])

In [ ]:
# print(train_input_texts[:10])
# print(train_input_tensor[:10])
# print(train_target_texts[:10])
# print(train_target_tensor[:10])
print(len(train_input_texts))
print(train_input_tensor.shape)
# print(len(train_target_texts))
# print(train_target_tensor.shape)

In [12]:
num_encoder_tokens = len(input_tokenizer.word_index)+1
num_decoder_tokens = len(target_tokenizer.word_index)+1
max_encoder_seq_length =  train_input_tensor.shape[1]
max_decoder_seq_length = train_target_tensor.shape[1]

In [13]:
print(input_tokenizer.word_index.keys())

dict_keys(['a', 'i', 'n', 'r', 'd', 't', 'u', 'h', 'e', 'l', 's', 'g', 'v', 'k', 'y', 'o', 'm', 'p', 'b', 'c', 'j', 'w', 'f', 'q', 'z', 'x'])


In [17]:
def build_layered_RNN_model(rnn_type,embedding_in_dim,embedding_out_dim,layers,dropout,inp_length,model_out_dim,initial_state):
   #input layer ; takes in tokenize input
  model_inputs = keras.Input(shape=( inp_length))
  #embedding layer
  embed = keras.layers.Embedding(embedding_in_dim, embedding_out_dim)(model_inputs)
  #will store output of last added layer so that we can add multiple layers
  last_layer_model = None
  if rnn_type == 'LSTM':
    #adding everything except the last LSTM layer, because in last layer return state=True
    for i in range(layers):
      layered_model = keras.layers.LSTM(model_out_dim, return_sequences=True,return_state=True,dropout=dropout)
      if i==0:
        model_layer_out,state_h, state_c = layered_model(embed,initial_state)
      else:
        model_layer_out,state_h, state_c = layered_model(last_layer_model,initial_state)
      last_layer_model = model_layer_out
    
    model_states = [state_h, state_c]
  elif rnn_type=='GRU':
    #adding everything except the last GRU layer, because in last layer return state=True    
    for i in range(layers):
      layered_model = keras.layers.GRU(model_out_dim, return_sequences=True,return_state=True,dropout=dropout)
      if i==0:
        model_layer_out,state = layered_model(embed,initial_state)
      else:
        model_layer_out,state = layered_model(last_layer_model,initial_state)
      last_layer_model = model_layer_out

    model_states = [state]
  elif rnn_type=='RNN':
    #adding everything except the last RNN layer, because in last layer return state=True
    for i in range(layers):      
      layered_model = keras.layers.SimpleRNN(model_out_dim, return_sequences=True,return_state=True,dropout=dropout)
      if i==0:
        model_layer_out,state = layered_model(embed,initial_state)
      else:
        model_layer_out,state = layered_model(last_layer_model,initial_state)
      last_layer_model = model_layer_out

    model_states = [state]

    return model_states,last_layer_model,model_inputs


In [18]:

#Build the model
def build_model(rnn_type,embedding_dim,encoder_layers,decoder_layers,dropout,latent_dim):
  encoder_states,encoder_outputs,encoder_inputs = build_layered_RNN_model(rnn_type,embedding_in_dim = num_encoder_tokens,embedding_out_dim = embedding_dim,layers = encoder_layers,dropout = dropout,inp_length = max_encoder_seq_length,model_out_dim = latent_dim,initial_state=None)

  decoder_states,decoder_outputs,decoder_inputs = build_layered_RNN_model(rnn_type,embedding_in_dim = num_decoder_tokens,embedding_out_dim = embedding_dim,layers = decoder_layers,dropout = dropout,inp_length = max_decoder_seq_length,model_out_dim = latent_dim,initial_state = encoder_states)
  
  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax",name='final')
  decoder_outputs = decoder_dense(decoder_outputs)

  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  return model